# "Recursive functions"
> "I explore recurive function functionality with some ML applications"

- author: Christopher Thiemann
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, ]
- hide: false
- search_exclude: true
- image: images/poisson.png

In [1]:
#hide
import warnings


import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols


import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

# supress warnings related to r
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings('ignore', category= FutureWarning)
warnings.filterwarnings('ignore', category= RRuntimeWarning)

#load the r interface
%load_ext rpy2.ipython
from rpy2.robjects import pandas2ri
pandas2ri.activate()

import rpy2.interactive as r
import rpy2.interactive.packages # this can take few seconds
rlib = r.packages.packages
r.packages.importr("utils")
rlib.utils.install_packages("tidyverse")
rlib.utils.install_packages("GGally")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/vectors.py:927: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '
R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/tidyverse_1.3.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 712837 bytes (696 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R

<rpy2.rinterface.NULLType object at 0x7fd6d3cbe808> [RTYPES.NILSXP]

In [2]:
#hide
# load r packages
%%R 
library(tidyverse)
library(GGally)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



Often I deal with methods which are based on a recursive structure. For example gradient descent, time series or in general recursive sequences like the fibonaci sequence. With recursive sequence I have in mind something of the form $x_n(x_{n-1})$ meaning the $x_n$ depends on its "past" value in some form. In gradient descent this looks for example as follows:

$\beta_n = \beta_{n-1} - \nu \nabla f(\beta_{n-1})$

Usually I implemented this with a simple for loop but today I want to implement this with the recursion capabilitys of python. ON the way I want  to develop a gernal framework on how to write recursive functions to solve arbitrary problems $x_n(x_{n-1}, x_{n-2}, x_{n-3}, ...)$

Lets start by writing down a recursive function for the factorial to get a feel of it. Remember, the factorial is defined as 

$n! = n * (n-1) * (n-2) ... * 1$

which can be written as 

$n! = n * (n-1)!$ $( x_n=n * x_{n-1})$

In [14]:
def fact(x):

    return x * fact( x - 1)

fact(5)

RecursionError: ignored

Hmm, what happend ? I clearly wrote a function which never terminates and hence runs forever. To avoid this python has set a limiting number of exceution times. This can be accesed with the folooing code

In [4]:
import sys
sys.getrecursionlimit()

1000

Of course our secquence has a natural end, namely when one sequence member is 1. So we note that we have to "break" out of the recursive function.

In [15]:
def fact(x):

    return x * fact( x - 1)

fact(5)

120

Much better! Now we are increasing the complexity a bit by considering a sequence which depents on two of its past values. A natural candidate is the fibonacci sequence which is defined as follows:

$x_n = x_{n-1} + x_{n-2}$ with $x_1 = 1$ and $x_0 = 0$

In [21]:
def fib(n):

    # x_1 = 1
    if n == 1:
        return 1

    # x_0 = 0
    if n == 0:
        return 0


    return fib(n - 1) + fib(n - 2)

fib(10)

55

Next I will implement gradient descent in an recursive way

In [11]:
def gradient_descent(estimate, old_estimate, gradient, X , y, step_size = .01, tol = .000001):
    
    if np.abs(old_estimate - estimate).sum() < tol:

        return estimate

    old_estimate = estimate.copy()

    estimate = estimate - step_size * gradient( estimate, X, y)

    return gradient_descent(estimate, old_estimate, gradient, X, y)


def gradient(estimate , X, y):

    n, p = X.shape
    return - 2 / n * X.T @ (y - X @ estimate)

X = np.random.randn(10000, 2)
beta = np.array([1, 2])

y = X @ beta + np.random.randn(10000)

gradient_descent(np.array([0,0]), np.array([1, 1]), gradient, X ,y)

array([1.00675273, 2.0022255 ])

In my opinion this is a very clean of doing gradient descent as it avoids the for loop. And with some small tweaks tihs could be generalized so that you simply pass a diferent gradient function say from a logist c regression and returns the solution.

## AR process

## Marcov Chain

## Helper Functions

## Plot for the Blog Post

## Sources

- Hello This is a markdown page {% cite signaltrain %}

## References

{% bibliography --cited %}